In [2]:
import os
import collections
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

In [3]:
# Define functions to read and process opcodes
def read_opcodes_from_file(filepath):
    with open(filepath, 'r') as file:
        opcodes = file.read().split()
    return opcodes

def create_opcode_frequency(opcodes):
    return collections.Counter(opcodes)

def process_directory(directory_path):
    files = os.listdir(directory_path)
    dataset = []
    for filename in files:
        filepath = os.path.join(directory_path, filename)
        opcodes = read_opcodes_from_file(filepath)
        frequency = create_opcode_frequency(opcodes)
        dataset.append((frequency, filename))
    return dataset

In [4]:
# Path to the new data folder
new_data_dir = '/home/ubuntu/Desktop/Test_Data'

# Process the new data directory
new_data = process_directory(new_data_dir)

# Load the previously saved model and vectorizer
with open('logistic_regression_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('vectorizer.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

# Vectorize the new data
new_frequencies = [freq for freq, _ in new_data]
X_new = vectorizer.transform(new_frequencies)

# Make predictions
predictions = model.predict(X_new)

# Create a DataFrame for the results
results = pd.DataFrame({
    'filename': [filename for _, filename in new_data],
    'prediction': predictions
})

# Map the predictions to human-readable labels
results['label'] = results['prediction'].map({1: 'malware', 0: 'goodware'})

# Save the results to a CSV file
results.to_csv('new_data_predictions.csv', index=False)

# Print the results
print(results)

# Count the number of malware and goodware
malware_count = results['label'].value_counts().get('malware', 0)
goodware_count = results['label'].value_counts().get('goodware', 0)

print(f'Number of malware samples: {malware_count}')
print(f'Number of goodware samples: {goodware_count}')

                                 filename  prediction     label
0    3962eefb27f952d30eb4a0834293ff01.txt           1   malware
1                            idlestat.txt           0  goodware
2                            dictfilt.txt           0  goodware
3                              lsattr.txt           0  goodware
4                                svok.txt           0  goodware
..                                    ...         ...       ...
271  da91ceb50720b4438ea0d283abb8dafb.txt           1   malware
272                         supervise.txt           0  goodware
273  cae9c4eab032e2234e796391d4a5608e.txt           1   malware
274                         gprostats.txt           0  goodware
275                          iptstate.txt           0  goodware

[276 rows x 3 columns]
Number of malware samples: 102
Number of goodware samples: 174
